## Notebook overview:
- In this notebook we load a csv files with links of pictures and tag them
- First we install the TFOD API
- Then we load the csv file out of csv directory into a data frame and manipulate them for our needs
- After that we create a list out of our Panda Series with links
- Then we need to define a function for loading the pictures from an url
- We predict each url(picture) and write the prediction to the dataframe
- Finally we can save the datframe in the csv format

In [1]:
#connect to our drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Cloning TFOD 2.0 Github needed for inferencing

In [2]:
cd /content/

/content


In [3]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 69341, done.
remote: Total 69341 (delta 0), reused 0 (delta 0), pack-reused 69341
Receiving objects: 100% (69341/69341), 577.37 MiB | 47.99 MiB/s, done.
Resolving deltas: 100% (48882/48882), done.


In [4]:
cd /content/models/research

/content/models/research


In [5]:
!protoc object_detection/protos/*.proto --python_out=.

In [6]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 30.47 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [7]:
cd cocoapi/PythonAPI

/content/models/research/cocoapi/PythonAPI


In [8]:
!make

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/models/research/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/n

### Install the Object Detection API

In [9]:
cd /content/models/research

/content/models/research


In [10]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.1 MB 8.6 MB/s 
     |████████████████████████████████| 352 kB 77.6 MB/s 
     |████████████████████████████████| 2.2 MB 66.6 MB/s 
     |████████████████████████████████| 23.4 MB 680 kB/s 
     |████████████████████████████████| 1.1 MB 64.9 MB/s 
     |████████████████████████████████| 47.7 MB 1.1 MB/s 
     |████████████████████████████████| 234 kB 78.3 MB/s 
     |████████████████████████████████| 4.9 MB 68.4 MB/s 
     |████████████████████████████████| 636 kB 73.2 MB/s 
     |████████████████████████████████| 43 kB 

In [11]:
# From within TensorFlow/models/research/
#test if tensorflow downloaded successfully
!python object_detection/builders/model_builder_tf2_test.py

2022-02-27 20:31:49.239754: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0227 20:31:49.705248 140629575182208 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.76s
I0227 20:31:50.005327 140629575182208 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.76s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.59s
I0227 20:31:50.598864 140629575182208 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_cr

## Import csv file, add needed columns and create a url_list 

In [9]:
cd /content/drive/MyDrive/Human_Detection_Tchibo

/content/drive/MyDrive/Human_Detection_Tchibo


In [10]:
import pandas as pd
import numpy as np

In [23]:
df_pics= pd.read_csv("tchibo_community_images.csv", header = None, names=["image", "user"] )

In [24]:
df_pics.head()

,image,user
0,https://assets-tch-live.s3.eu-central-1.amazon...,d333f3791580c59e1fb803ced599da40
1,https://assets-tch-live.s3.eu-central-1.amazon...,bc27b92c5a7625a04394e19fb6b53b5f
2,https://assets-tch-live.s3.eu-central-1.amazon...,badc9b7866e336a9ae6e0db58f4ba7d1
3,https://assets-tch-live.s3.eu-central-1.amazon...,97c5f42c0eafe066300d47bddd0c14dc
4,https://assets-tch-live.s3.eu-central-1.amazon...,47567a5f001ab3a178aebd9e58a352df


In [13]:
df_pics.user.value_counts()

2e95251e28153ff771579f7c2330368d    7
badc9b7866e336a9ae6e0db58f4ba7d1    5
6845c7f8c91ce1f985fecfc39a03c33a    5
67b21469e20c03ba49af6d78bffb46fc    5
e9773d68cc2975c1dfca9b9e9fa2a871    4
                                   ..
3076d12136e4a3a18271f7b13110f39c    1
2e95a3d662ed51e6a2aa7917419c0825    1
0be8cf3c99316e2284ac832681beeaf9    1
d2d589ab7b56c0cb5ce072821747fc7f    1
a086e4344da34ad99e90426d61827160    1
Name: user, Length: 299, dtype: int64

In [14]:
url_list= df_pics.image.tolist()

--> hier können wir dann noch eine kleine explorative datenanalyse machen. wer postet am meisten Tchibofotos ... und wer welche mit hohen confidence score (gut zum Marketing verwenden) --> groupby...

##Load the model

If there are problems with cv2 ("registerMatetype" or something like that) run the cell below and restart runtime

In [ ]:
#upgrade specific opencv liabaries to have all opencv libaries up to date
!pip uninstall opencv-python
!pip install opencv-python
!pip uninstall opencv-contrib-python
!pip install opencv-contrib-python
!pip uninstall opencv-python-headless
!pip install opencv-python-headless

In [ ]:
# load the model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

path_to_saved_model = "/content/drive/MyDrive/training_demo/new_exported_models/Exported_centernet_512_updated/saved_model"

print('Loading model...', end='')
start_time = time.time()

# load saved model 
detect_fn = tf.saved_model.load(path_to_saved_model)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

## Predict each url and write the first detection class and belonging confidence score to dataframe with urls

In [ ]:
# LOAD LABEL MAP DATA FOR PLOTTING
# PROVIDE PATH TO LABEL MAp

path_to_labels = '/content/drive/MyDrive/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels,
                                                                    use_display_name=True)
#import all packages
import cv2 
from google.colab.patches import cv2_imshow                                                                   
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
from skimage import io
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

#define function reading image from url 
def load_image_from_url(url):
  return io.imread(url)

#define classification threshold. Determines which predictions are written to csv file
classification_threshold = 0.5


#iterate through url list and display each detection
i=0
for picture in url_list:

  image= load_image_from_url(picture) #--> that does not work yet
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #image_expanded = np.expand_dims(image_rgb, axis=0)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.


  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}

  detections['detection_classes'] = np.where(detections['detection_classes'] == 1, 'Tchibo Logo', 'Face')

  n_detection = 0
  for detection in detections['detection_scores']:
    if detections['detection_scores'][n_detection] >= classification_threshold:
      df_pics.loc[df_pics['image'] == picture, 'detectionclass_' + str(n_detection)] = \
      detections['detection_classes'][n_detection]
      df_pics.loc[df_pics['image'] == picture, 'detectionscore_' + str(n_detection)] = \
      detections['detection_scores'][n_detection]
      '''
      #write the bounding box coordinates to csv file
      df_pics.loc[df_pics['image'] == picture, 'x_min_' + str(n_detection)] = \
      detections['detection_boxes'][n_detection][0] * 1000
      df_pics.loc[df_pics['image'] == picture, 'y_min_' + str(n_detection)] = \
      detections['detection_boxes'][n_detection][1] * 1000
      df_pics.loc[df_pics['image'] == picture, 'x_max_' + str(n_detection)] = \
      detections['detection_boxes'][n_detection][2] * 1000
      df_pics.loc[df_pics['image'] == picture, 'y_max_' + str(n_detection)] = \
      detections['detection_boxes'][n_detection][3] * 1000
      '''
      n_detection += 1
    else:
      n_detection += 1



Display the detection class and confidence scroe to compare to dataframe data. 
Checks if data was written correctly to dataframe

In [ ]:
# LOAD LABEL MAP DATA FOR PLOTTING
# PROVIDE PATH TO LABEL MAP
path_to_labels = '/content/drive/MyDrive/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels,
                                                                    use_display_name=True)
#import all packages
import cv2 
from google.colab.patches import cv2_imshow                                                                   
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
from skimage import io
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

#define function reading image from url 
def load_image_from_url(url):
  return io.imread(url)

#iterate through image path list and display each detection
i=0
for urls in url_list:
  if i >4:
    break
  i+=1
  image= load_image_from_url(urls) #--> that does not work yet
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_with_detections = image.copy()

  # set min_score_thresh based on your minimum thrshold for detection 
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.25,
        agnostic_mode=False)

  # display output image 
  cv2_imshow(image_with_detections)
  

In [ ]:
df_pics.head(20)

### Write dataframe to csv and download csv

In [ ]:
df.to_csv(r"/content/drive/MyDrive/training_demo/images/tcompics_labled.csv", index=False)